In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1442, with_units=False)
Hist_table


Days,Entries
IntegerAndSexagesimal,Historical
1 ;,24 ; 23
2 ;,01s 18 ; 46
3 ;,02s 13 ; 09
4 ;,03s 07 ; 33
5 ;,04s 01 ; 54
6 ;,04s 26 ; 17
7 ;,05s 20 ; 40
8 ;,06s 15 ; 03
9 ;,07s 09 ; 46


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(31):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [4]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=2*Historical(Sexagesimal("12;11,26,41,37,51,50,39"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(31):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
        Test_recomp[i][1]=daily_mean*Hist_table[i][0]%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=1):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=60*(Hist_table[i][1]-Test_recomp[i][1])

In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(1.2580645161290323, nan)

In [6]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Days,
1 ;,0.000000
2 ;,0.000000
3 ;,0.000000
4 ;,1.000000
5 ;,0.000000
6 ;,0.000000
7 ;,0.000000
8 ;,0.000000
9 ;,20.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Lunar mean center days.csv')
B.to_csv('Lunar mean center days.zip', index=False, compression=compression_opts)